# Part 1
## Comments Extraction

In [66]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [67]:
import os
os.chdir('/content/drive/My Drive/Analytic-Steps-blogs')

## Install the relevant libraries

In [68]:
pip install google-api-python-client

In [69]:
pip install google-auth google-auth-oauthlib google-auth-httplib2

In [72]:
pip install langdetect

In [73]:
pip install pandas 

In [74]:
pip install demoji

In [76]:
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
import pandas as pd
import demoji
from langdetect import detect
import re   # regular expression

In [77]:
import pandas as pd
import os

### provide path to the `client_secret.json` file which will be useful inthe authorization

In [78]:
CLIENT_SECRETS_FILE = "/content/drive/My Drive/Analytic-Steps-blogs/client_secret.json"

In [79]:
SCOPES = ['https://www.googleapis.com/auth/youtube.force-ssl']
API_SERVICE_NAME = 'youtube'
API_VERSION = 'v3'

In [80]:
def get_authenticated_service():
  flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRETS_FILE, SCOPES)
  credentials = flow.run_console()
  return build(API_SERVICE_NAME, API_VERSION, credentials = credentials)

In [81]:
os.environ['OAUTHLIB_INSECURE_TRANSPORT'] = '1'
service = get_authenticated_service()

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=1067751526010-uhjt4rd2c451rg6kk6f50onjoqnhliql.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.force-ssl&state=u7vPbUynH9RvGwRzMijxOQqx2CnZ0K&prompt=consent&access_type=offline
Enter the authorization code: 4/1AfDhmrgT4lp5vVZ-xiAccWeqoQPqT9mvqN5cteC3x-6Kjzkuh_-rzJPoTv4


### Set the search query i.e. video of which comments will be extracted

In [130]:
# query = "Sia - Cheap Thrills (Lyric Video) ft. Sean Paul"
# query = "Clean Bandit - Rockabye (feat. Sean Paul & Anne-Marie) [Official Video]"
query = "Telecommunications & Network Engineering Degree | MS Telecom At SMU"

In [131]:
query_results = service.search().list(part = 'snippet',q = query,
                                      order = 'relevance', 
                                      type = 'video',
                                      relevanceLanguage = 'en',
                                      safeSearch = 'moderate').execute()

In [132]:
query_results['items']

[{'etag': '4Eredbn2EbLiQMMLNt_voafUWgQ',
  'id': {'kind': 'youtube#video', 'videoId': 'lkBJdYWh9LU'},
  'kind': 'youtube#searchResult',
  'snippet': {'channelId': 'UCDLoaI_zOwy_vCPnie9fCXQ',
   'channelTitle': 'Chai & Coaching',
   'description': 'Jay Shah shares about his journey studying his masters degree of Telecommunications and Network Engineering at Southern Methodist University. Learn about ...',
   'liveBroadcastContent': 'none',
   'publishTime': '2020-11-04T17:15:11Z',
   'publishedAt': '2020-11-04T17:15:11Z',
   'thumbnails': {'default': {'height': 90,
     'url': 'https://i.ytimg.com/vi/lkBJdYWh9LU/default.jpg',
     'width': 120},
    'high': {'height': 360,
     'url': 'https://i.ytimg.com/vi/lkBJdYWh9LU/hqdefault.jpg',
     'width': 480},
    'medium': {'height': 180,
     'url': 'https://i.ytimg.com/vi/lkBJdYWh9LU/mqdefault.jpg',
     'width': 320}},
   'title': 'Telecommunications &amp; Network Engineering Degree | MS Telecom At SMU'}},
 {'etag': 'SEcAPqFZPPbZOsuZ8rg5

### Extract video details i.e. videoID, channel Name, videoTitle, videoDescription

In [133]:
video_id = []
channel = []
video_title = []
video_desc = []
for item in query_results['items']:
    video_id.append(item['id']['videoId'])
    channel.append(item['snippet']['channelTitle'])
    video_title.append(item['snippet']['title'])
    video_desc.append(item['snippet']['description'])

[link text](**https://**)We only wants to work with the one video so select the first element in list and save it

In [134]:
video_id = video_id[0]
channel = channel[0]
video_title = video_title[0]
video_desc = video_desc[0]


### Extract Comments
In the next step we will extract the comments

In [135]:
video_id_pop = []
channel_pop = []
video_title_pop = []
video_desc_pop = []
comments_pop = []
comment_id_pop = []
reply_count_pop = []
like_count_pop = []


comments_temp = []
comment_id_temp = []
reply_count_temp = []
like_count_temp = []


In [136]:
nextPage_token = None

while 1:
  response = service.commentThreads().list(
                    part = 'snippet',
                    videoId = video_id,
                    maxResults = 100, 
                    order = 'relevance', 
                    textFormat = 'plainText',
                    pageToken = nextPage_token
                    ).execute()


  nextPage_token = response.get('nextPageToken')
  for item in response['items']:
      comments_temp.append(item['snippet']['topLevelComment']['snippet']['textDisplay'])
      comment_id_temp.append(item['snippet']['topLevelComment']['id'])
      reply_count_temp.append(item['snippet']['totalReplyCount'])
      like_count_temp.append(item['snippet']['topLevelComment']['snippet']['likeCount'])
      comments_pop.extend(comments_temp)
      comment_id_pop.extend(comment_id_temp)
      reply_count_pop.extend(reply_count_temp)
      like_count_pop.extend(like_count_temp)
        
      video_id_pop.extend([video_id]*len(comments_temp))
      channel_pop.extend([channel]*len(comments_temp))
      video_title_pop.extend([video_title]*len(comments_temp))
      video_desc_pop.extend([video_desc]*len(comments_temp))

  if nextPage_token is  None:
    break

# print(allVideos)
# 

In [150]:
comments_pop

['☕️Chai Question: What degree do you want to pursue in America?☕️\n\nMultiple Visa Interview Rejections: https://youtu.be/eyDHxDRoI-I',
 '☕️Chai Question: What degree do you want to pursue in America?☕️\n\nMultiple Visa Interview Rejections: https://youtu.be/eyDHxDRoI-I',
 'Masters in Biomedical Engineering..',
 '☕️Chai Question: What degree do you want to pursue in America?☕️\n\nMultiple Visa Interview Rejections: https://youtu.be/eyDHxDRoI-I',
 'Masters in Biomedical Engineering..',
 "Thank you so soooooooooo much , I'll be pursuing masters in the same field ,......also the same with NEU Boston, \n\nI was waiting for this video from a very long time,\nThanks rob, thanks Jay",
 '☕️Chai Question: What degree do you want to pursue in America?☕️\n\nMultiple Visa Interview Rejections: https://youtu.be/eyDHxDRoI-I',
 'Masters in Biomedical Engineering..',
 "Thank you so soooooooooo much , I'll be pursuing masters in the same field ,......also the same with NEU Boston, \n\nI was waiting fo

### As per now comments have been extracted from youtube video and now we will save the comments into pandas dataframe and then write in csv file

In [147]:


output_dict = {
        'Channel': channel_pop,
        'Video Title': video_title_pop,
        'Video Description': video_desc_pop,
        'Video ID': video_id_pop,
        'Comment': comments_pop,
        'Comment ID': comment_id_pop,
        'Replies': reply_count_pop,
        'Likes': like_count_pop,
        }

output_df = pd.DataFrame(output_dict, columns = output_dict.keys())

In [148]:
output_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Channel            10 non-null     object
 1   Video Title        10 non-null     object
 2   Video Description  10 non-null     object
 3   Video ID           10 non-null     object
 4   Comment            10 non-null     object
 5   Comment ID         10 non-null     object
 6   Replies            10 non-null     int64 
 7   Likes              10 non-null     int64 
dtypes: int64(2), object(6)
memory usage: 768.0+ bytes


In [149]:
output_df.head()

,Channel,Video Title,Video Description,Video ID,Comment,Comment ID,Replies,Likes
0,Chai & Coaching,Telecommunications &amp; Network Engineering D...,Jay Shah shares about his journey studying his...,lkBJdYWh9LU,☕️Chai Question: What degree do you want to pu...,UgxD4O4SyXyAeypy7bl4AaABAg,0,1
1,Chai & Coaching,Telecommunications &amp; Network Engineering D...,Jay Shah shares about his journey studying his...,lkBJdYWh9LU,☕️Chai Question: What degree do you want to pu...,UgxD4O4SyXyAeypy7bl4AaABAg,0,1
2,Chai & Coaching,Telecommunications &amp; Network Engineering D...,Jay Shah shares about his journey studying his...,lkBJdYWh9LU,Masters in Biomedical Engineering..,Ugzwmy9YzWRCFGMEI0h4AaABAg,1,1
3,Chai & Coaching,Telecommunications &amp; Network Engineering D...,Jay Shah shares about his journey studying his...,lkBJdYWh9LU,☕️Chai Question: What degree do you want to pu...,UgxD4O4SyXyAeypy7bl4AaABAg,0,1
4,Chai & Coaching,Telecommunications &amp; Network Engineering D...,Jay Shah shares about his journey studying his...,lkBJdYWh9LU,Masters in Biomedical Engineering..,Ugzwmy9YzWRCFGMEI0h4AaABAg,1,1


### Observe the above dataframe output, many comments are repeating itself,so lets check it how many duplicate `comments` are present inthe dataframe.
Use `pandas.dataframe.duplicated()` to get the duplicates and `pandas.dataframe.drop_duplicates()` to drop all those duplicates

In [141]:
duplicates = output_df[output_df.duplicated("Comment ID")]
print("Count of duplicat comments in dataframe", duplicates.shape[0])

Count of duplicat comments in dataframe 6


In [142]:
print("count of unique comments in dataframe ",output_df.shape[0]- duplicates.shape[0])

count of unique comments in dataframe  4


In [143]:
unique_df = output_df.drop_duplicates(subset=['Comment'])

In [144]:
unique_df.shape

(4, 8)

In the following dataframe all the comments are unique i.e. no comment appears more than once

In [145]:
unique_df.head()

,Channel,Video Title,Video Description,Video ID,Comment,Comment ID,Replies,Likes
0,Chai & Coaching,Telecommunications &amp; Network Engineering D...,Jay Shah shares about his journey studying his...,lkBJdYWh9LU,☕️Chai Question: What degree do you want to pu...,UgxD4O4SyXyAeypy7bl4AaABAg,0,1
2,Chai & Coaching,Telecommunications &amp; Network Engineering D...,Jay Shah shares about his journey studying his...,lkBJdYWh9LU,Masters in Biomedical Engineering..,Ugzwmy9YzWRCFGMEI0h4AaABAg,1,1
5,Chai & Coaching,Telecommunications &amp; Network Engineering D...,Jay Shah shares about his journey studying his...,lkBJdYWh9LU,"Thank you so soooooooooo much , I'll be pursui...",UgyEwpGS3mvj3SswC2J4AaABAg,0,0
9,Chai & Coaching,Telecommunications &amp; Network Engineering D...,Jay Shah shares about his journey studying his...,lkBJdYWh9LU,Masters in Data Science,UgyCJr4OjRPVQ63MRed4AaABAg,1,1


## save the dataframe into csv file

In [146]:
unique_df.to_csv("extraced_comments.csv",index = False)

# Part 2
## Data Cleaning

Install `demoji` library which will be used to remove emojis from the comments

In [151]:
pip install demoji

In [152]:
comments = pd.read_csv('extraced_comments.csv')

In [153]:
comments.head()

,Channel,Video Title,Video Description,Video ID,Comment,Comment ID,Replies,Likes
0,Chai & Coaching,Telecommunications &amp; Network Engineering D...,Jay Shah shares about his journey studying his...,lkBJdYWh9LU,☕️Chai Question: What degree do you want to pu...,UgxD4O4SyXyAeypy7bl4AaABAg,0,1
1,Chai & Coaching,Telecommunications &amp; Network Engineering D...,Jay Shah shares about his journey studying his...,lkBJdYWh9LU,Masters in Biomedical Engineering..,Ugzwmy9YzWRCFGMEI0h4AaABAg,1,1
2,Chai & Coaching,Telecommunications &amp; Network Engineering D...,Jay Shah shares about his journey studying his...,lkBJdYWh9LU,"Thank you so soooooooooo much , I'll be pursui...",UgyEwpGS3mvj3SswC2J4AaABAg,0,0
3,Chai & Coaching,Telecommunications &amp; Network Engineering D...,Jay Shah shares about his journey studying his...,lkBJdYWh9LU,Masters in Data Science,UgyCJr4OjRPVQ63MRed4AaABAg,1,1


In [154]:
comments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Channel            4 non-null      object
 1   Video Title        4 non-null      object
 2   Video Description  4 non-null      object
 3   Video ID           4 non-null      object
 4   Comment            4 non-null      object
 5   Comment ID         4 non-null      object
 6   Replies            4 non-null      int64 
 7   Likes              4 non-null      int64 
dtypes: int64(2), object(6)
memory usage: 384.0+ bytes


### Remove Emojis 
In this step we will remove emojis from the comments using `demoji`

In [155]:
import demoji

In [156]:
comments = pd.read_csv('extraced_comments.csv')

In [157]:
## this step is necessary as it will download the latest emojis if updaed so as to remove all from the given texts

demoji.download_codes()

... OK (Got response in 0.13 seconds)
Writing emoji data to /root/.demoji/codes.json ...
... OK


`demoji.findall()` will gives all the emojis present in code with their tags as seen in below<br>
`demoji.replace()` will be useful in replacing emojis from text.

In [158]:
comments['clean_comments'] = comments['Comment'].apply(lambda x: demoji.replace(x,""))

In [159]:
comments.head()

,Channel,Video Title,Video Description,Video ID,Comment,Comment ID,Replies,Likes,clean_comments
0,Chai & Coaching,Telecommunications &amp; Network Engineering D...,Jay Shah shares about his journey studying his...,lkBJdYWh9LU,☕️Chai Question: What degree do you want to pu...,UgxD4O4SyXyAeypy7bl4AaABAg,0,1,️Chai Question: What degree do you want to pur...
1,Chai & Coaching,Telecommunications &amp; Network Engineering D...,Jay Shah shares about his journey studying his...,lkBJdYWh9LU,Masters in Biomedical Engineering..,Ugzwmy9YzWRCFGMEI0h4AaABAg,1,1,Masters in Biomedical Engineering..
2,Chai & Coaching,Telecommunications &amp; Network Engineering D...,Jay Shah shares about his journey studying his...,lkBJdYWh9LU,"Thank you so soooooooooo much , I'll be pursui...",UgyEwpGS3mvj3SswC2J4AaABAg,0,0,"Thank you so soooooooooo much , I'll be pursui..."
3,Chai & Coaching,Telecommunications &amp; Network Engineering D...,Jay Shah shares about his journey studying his...,lkBJdYWh9LU,Masters in Data Science,UgyCJr4OjRPVQ63MRed4AaABAg,1,1,Masters in Data Science


Use `langdetect` library to detect language of the comment

In [160]:
from textblob import TextBlob

In [161]:

from langdetect import detect

Now add new feature `language` inthe dataframe which signify the language of particular comment

In [162]:
comments['language'] = 0

In [163]:
count = 0
for i in range(0,len(comments)):


  temp = comments['clean_comments'].iloc[i]
  count += 1
  try:
    comments['language'].iloc[i] = detect(temp)
  except:
    comments['language'].iloc[i] = "error"

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [164]:
                                                                                                  comments.head()

,Channel,Video Title,Video Description,Video ID,Comment,Comment ID,Replies,Likes,clean_comments,language
0,Chai & Coaching,Telecommunications &amp; Network Engineering D...,Jay Shah shares about his journey studying his...,lkBJdYWh9LU,☕️Chai Question: What degree do you want to pu...,UgxD4O4SyXyAeypy7bl4AaABAg,0,1,️Chai Question: What degree do you want to pur...,en
1,Chai & Coaching,Telecommunications &amp; Network Engineering D...,Jay Shah shares about his journey studying his...,lkBJdYWh9LU,Masters in Biomedical Engineering..,Ugzwmy9YzWRCFGMEI0h4AaABAg,1,1,Masters in Biomedical Engineering..,en
2,Chai & Coaching,Telecommunications &amp; Network Engineering D...,Jay Shah shares about his journey studying his...,lkBJdYWh9LU,"Thank you so soooooooooo much , I'll be pursui...",UgyEwpGS3mvj3SswC2J4AaABAg,0,0,"Thank you so soooooooooo much , I'll be pursui...",en
3,Chai & Coaching,Telecommunications &amp; Network Engineering D...,Jay Shah shares about his journey studying his...,lkBJdYWh9LU,Masters in Data Science,UgyCJr4OjRPVQ63MRed4AaABAg,1,1,Masters in Data Science,en


 Check for the count of english comments

In [165]:
comments[comments['language']=='en']['language'].value_counts()

en    4
Name: language, dtype: int64

Save the english comments into csv file

In [166]:
english_comm = comments[comments['language'] == 'en']

In [167]:
english_comm.to_csv("english_comments.csv",index = False)

# Part 2.2

### Load the dataset which contain only english comments

In [168]:
en_comments = pd.read_csv('english_comments.csv',encoding='utf8',error_bad_lines=False)

In [169]:
en_comments.head()

,Channel,Video Title,Video Description,Video ID,Comment,Comment ID,Replies,Likes,clean_comments,language
0,Chai & Coaching,Telecommunications &amp; Network Engineering D...,Jay Shah shares about his journey studying his...,lkBJdYWh9LU,☕️Chai Question: What degree do you want to pu...,UgxD4O4SyXyAeypy7bl4AaABAg,0,1,️Chai Question: What degree do you want to pur...,en
1,Chai & Coaching,Telecommunications &amp; Network Engineering D...,Jay Shah shares about his journey studying his...,lkBJdYWh9LU,Masters in Biomedical Engineering..,Ugzwmy9YzWRCFGMEI0h4AaABAg,1,1,Masters in Biomedical Engineering..,en
2,Chai & Coaching,Telecommunications &amp; Network Engineering D...,Jay Shah shares about his journey studying his...,lkBJdYWh9LU,"Thank you so soooooooooo much , I'll be pursui...",UgyEwpGS3mvj3SswC2J4AaABAg,0,0,"Thank you so soooooooooo much , I'll be pursui...",en
3,Chai & Coaching,Telecommunications &amp; Network Engineering D...,Jay Shah shares about his journey studying his...,lkBJdYWh9LU,Masters in Data Science,UgyCJr4OjRPVQ63MRed4AaABAg,1,1,Masters in Data Science,en


In [170]:
en_comments.shape

(4, 10)

### Remove brackets & special characters using Regular Expressions

Now first write the grammar to remove special characters from the comments

In [171]:
regex = r"[^0-9A-Za-z'\t]"

In [172]:
copy = en_comments.copy()

In [173]:
import re

In [174]:
copy['reg'] = copy['clean_comments'].apply(lambda x:re.findall(regex,x))
copy['regular_comments'] = copy['clean_comments'].apply(lambda x:re.sub(regex,"  ",x))

In [175]:
copy.head()

,Channel,Video Title,Video Description,Video ID,Comment,Comment ID,Replies,Likes,clean_comments,language,reg,regular_comments
0,Chai & Coaching,Telecommunications &amp; Network Engineering D...,Jay Shah shares about his journey studying his...,lkBJdYWh9LU,☕️Chai Question: What degree do you want to pu...,UgxD4O4SyXyAeypy7bl4AaABAg,0,1,️Chai Question: What degree do you want to pur...,en,"[️, , :, , , , , , , , , , ?, ️, \n,...",Chai Question What degree do you wan...
1,Chai & Coaching,Telecommunications &amp; Network Engineering D...,Jay Shah shares about his journey studying his...,lkBJdYWh9LU,Masters in Biomedical Engineering..,Ugzwmy9YzWRCFGMEI0h4AaABAg,1,1,Masters in Biomedical Engineering..,en,"[ , , , ., .]",Masters in Biomedical Engineering
2,Chai & Coaching,Telecommunications &amp; Network Engineering D...,Jay Shah shares about his journey studying his...,lkBJdYWh9LU,"Thank you so soooooooooo much , I'll be pursui...",UgyEwpGS3mvj3SswC2J4AaABAg,0,0,"Thank you so soooooooooo much , I'll be pursui...",en,"[ , , , , , ,, , , , , , , , , , ...",Thank you so soooooooooo much I'll b...
3,Chai & Coaching,Telecommunications &amp; Network Engineering D...,Jay Shah shares about his journey studying his...,lkBJdYWh9LU,Masters in Data Science,UgyCJr4OjRPVQ63MRed4AaABAg,1,1,Masters in Data Science,en,"[ , , ]",Masters in Data Science


## As now we are done with all the preprocessing part of the data.
Now the next step will be sentiment analysis so for that we will make a new dataframe containing only `Video ID`, `Comment ID`,`regular_comments`
<br> after that will write into a new csv file

In [176]:
dataset = copy[['Video ID','Comment ID','regular_comments']].copy()

In [177]:
dataset.head()

,Video ID,Comment ID,regular_comments
0,lkBJdYWh9LU,UgxD4O4SyXyAeypy7bl4AaABAg,Chai Question What degree do you wan...
1,lkBJdYWh9LU,Ugzwmy9YzWRCFGMEI0h4AaABAg,Masters in Biomedical Engineering
2,lkBJdYWh9LU,UgyEwpGS3mvj3SswC2J4AaABAg,Thank you so soooooooooo much I'll b...
3,lkBJdYWh9LU,UgyCJr4OjRPVQ63MRed4AaABAg,Masters in Data Science


In [178]:
dataset = dataset.rename(columns = {"regular_comments":"comments"})

In [179]:
dataset.head()

,Video ID,Comment ID,comments
0,lkBJdYWh9LU,UgxD4O4SyXyAeypy7bl4AaABAg,Chai Question What degree do you wan...
1,lkBJdYWh9LU,Ugzwmy9YzWRCFGMEI0h4AaABAg,Masters in Biomedical Engineering
2,lkBJdYWh9LU,UgyEwpGS3mvj3SswC2J4AaABAg,Thank you so soooooooooo much I'll b...
3,lkBJdYWh9LU,UgyCJr4OjRPVQ63MRed4AaABAg,Masters in Data Science


In [180]:
dataset.to_csv("Dataset.csv",index = False)